In [9]:
from Reimplementation.data.DeepCDRDataModule import DeepCDRDataset
from torch_geometric.datasets import FakeDataset
from torch_geometric.data import Batch, Data
import torch_geometric.data
from pytorch_lightning.loggers import TensorBoardLogger
from torch.optim import SGD
from Reimplementation.util.utils import *
from Reimplementation.util.config import *
import torch
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import random_split, TensorDataset, DataLoader
from torch import nn

%load_ext tensorboard
pl.seed_everything(0)

GCN = ConvType.GCN
GAT = ConvType.GAT
tanh = ActivationFunc.Tanh
relu = ActivationFunc.ReLU
f_tanh = FuncActivationFunc.Tanh
f_relu = FuncActivationFunc.ReLU

dpath = '../data'
drug_info_file = '%s/GDSC/1.Drug_listMon Jun 24 09_00_55 2019.csv' % dpath
cell_line_info_file = '%s/CCLE/Cell_lines_annotations_20181226.txt' % dpath
drug_feature_file = '%s/GDSC/drug_graph_feat' % dpath
genomic_mutation_file = '%s/CCLE/genomic_mutation_34673_demap_features.csv' % dpath
cancer_response_exp_file = '%s/CCLE/GDSC_IC50.csv' % dpath
gene_expression_file = '%s/CCLE/genomic_expression_561celllines_697genes_demap_features.csv' % dpath
methylation_file = '%s/CCLE/genomic_methylation_561celllines_808genes_demap_features.csv' % dpath
raw_files = NeededFiles(drug_info_file, cell_line_info_file, drug_feature_file, genomic_mutation_file,
                        cancer_response_exp_file, gene_expression_file, methylation_file)
graph_dataset = DeepCDRDataset(raw_files, 100)


class TestModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.cnn = create_1d_cnn(Conv1DConfig([1, 30, 90, 200], [10, 10, 10], [tanh, relu, relu], [10, 10, 10], False))
        self.fc_input = create_fcn(FCConfig([256], [relu], output=128, p_dropout=0, use_bn=False))
        self.reg = create_fcn(FCConfig([100, 50, 10], [relu, relu, relu], output=1, p_dropout=0, use_bn=False))

    def forward(self, x_1, x_2):
        x_1_feat = self.cnn(x_1)
        x_1_feat = nn.Flatten()(x_1_feat)

        x_2 = self.fc_input(x_2)

        x_feat = torch.cat((x_1_feat, x_2), dim=-1)

        output = self.reg(x_feat)
        return output

    def training_step(self, batch):
        x_1, x_2, y = batch
        loss = F.mse_loss(self(x_1, x_2), y)
        self.log("train loss", loss, logger=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.01)

    def train_dataloader(self):
        x_1 = torch.ones(1, 1, 34673)
        x_2 = torch.ones(1, 808)
        y = torch.zeros(1)
        dataset = TensorDataset(x_1, x_2, y)
        dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
        return dataloader

class GNNTest(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.gnn = create_gnn(GNNConfig([75, 512, 100], [GCN, GCN], [f_relu, f_relu], False))
        self.reg = create_fcn(FCConfig([100, 50], [relu, relu], output=1, use_bn=False))

    def forward(self, x, edge_index):
        feat = self.gnn(x.squeeze(), edge_index.squeeze())
        x_feat = torch.cat((torch.flatten(feat), torch.ones(100)), dim=-1)

        output = self.reg(x_feat)
        return output

    def training_step(self, batch):
        x, edge_index, y = batch
        loss = F.mse_loss(self(x, edge_index), y)
        self.log("train loss", loss, logger=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.01)

    def train_dataloader(self):
        x = graph_dataset[0][0].x[None, :]
        edge_index = graph_dataset[0][0].edge_index[None, :]
        y = torch.zeros(1)
        dataset = TensorDataset(x, edge_index, y)
        dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
        return dataloader

model = GNNTest()
x, edge_index, y = next(iter(model.train_dataloader()))
model.forward(x, edge_index)

Global seed set to 0


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


/home/user/PycharmProjects/DeepCDR/venv/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


tensor([-0.0633], grad_fn=<AddBackward0>)

In [10]:
logger=TensorBoardLogger(save_dir="test_log", name="FCTest")
trainer = pl.Trainer(logger=logger, max_epochs=300, overfit_batches=1, log_every_n_steps=5)
trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.

  | Name | Type              | Params
-------------------------------------------
0 | gnn  | Sequential_ce74e3 | 90.2 K
1 | reg  | Sequential        | 1.0 M 
-------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.422     Total estimated model params size (MB)
/home/user/PycharmProjects/DeepCDR/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/user/PycharmProjects/DeepCD

Training: 0it [00:00, ?it/s]

2022-06-06 15:04:05.998228: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-06 15:04:05.998294: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Using TensorFlow backend.


In [44]:
%reload_ext tensorboard
%tensorboard --logdir test_log/

Launching TensorBoard...